# 1 - Introduction to Random Forests

[Course Notebook on Github](https://github.com/fastai/fastai/blob/master/courses/ml1/lesson1-rf.ipynb)
### Notes
My goal is not to copy what it is taught in the course and in the notebook. Its just various notes and trying things for my own.

#### Imports

In [59]:
# import
%load_ext autoreload
%autoreload 2

%matplotlib inline
# libs
import os
import pandas as pd
import numpy as np
import re, time
from sklearn.ensemble import RandomForestRegressor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# ??display !!!

In [6]:
PATH = './datasets/bluebook_bulldozers/'

In [8]:
!ls {PATH} # bash command

'Data Dictionary.xlsx'		    Train.7z	        Valid.7z
 Machine_Appendix.csv		    TrainAndValid.7z    Valid.csv
 median_benchmark.csv		    TrainAndValid.csv   ValidSolution.csv
 random_forest_benchmark_test.csv   TrainAndValid.zip   Valid.zip
 Test.csv			    Train.zip


In [10]:
!head datasets/bluebook_bulldozers/Train.csv

SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
1139246,66000,999089,3157,121,3,2004,68,Low,11/16/2006 0:00,521D,521,D,,,,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,,EROPS w AC,None or Unspecified,,None or Unspecified,,,,,,,,2 Valve,,,,,None or Unspecified,None or Unspecified,,,,,,,,,,,,,Standard,Conventional
1139248,57000,117657,77,121,3,1996,4640,Lo

In [42]:
df_raw = pd.read_csv(f'{PATH}Train.csv', low_memory=False, parse_dates=['saledate'])#implicity datetype)

In [14]:
def display_all(df):
    with pd.option_context('display.max_rows', 1000):
        with pd.option_context('display.max_columns', 1000):
            display(df)

In [15]:
display_all(df_raw.tail().transpose())

401120  \
SalesID                                                             6333336   
SalePrice                                                             10500   
MachineID                                                           1840702   
ModelID                                                               21439   
datasource                                                              149   
auctioneerID                                                              1   
YearMade                                                               2005   
MachineHoursCurrentMeter                                                NaN   
UsageBand                                                               NaN   
saledate                                                2011-11-02 00:00:00   
fiModelDesc                                                           35NX2   
fiBaseModel                                                              35   
fiSecondaryDesc                                                          NX   
fiModelSeries                                                             2   
fiModelDescriptor                                                       NaN   
ProductSize                                                            Mini   
fiProductClassDesc        Hydraulic Excavator, Track - 3.0 to 4.0 Metric...   
state                                                              Maryland   
ProductGroup                                                            TEX   
ProductGroupDesc                                           Track Excavators   
Drive_System                                                            NaN   
Enclosure                                                             EROPS   
Forks                                                                   NaN   
Pad_Type                                                                NaN   
Ride_Control                                                            NaN   
Stick                                                                   NaN   
Transmission                                                            NaN   
Turbocharged                                                            NaN   
Blade_Extension                                                         NaN   
Blade_Width                                                             NaN   
Enclosure_Type                                                          NaN   
Engine_Horsepower                                                       NaN   
Hydraulics                                                        Auxiliary   
Pushblock                                                               NaN   
Ripper                                                                  NaN   
Scarifier                                                               NaN   
Tip_Control                                                             NaN   
Tire_Size                                                               NaN   
Coupler                                                 None or Unspecified   
Coupler_System                                                          NaN   
Grouser_Tracks                                                          NaN   
Hydraulics_Flow                                                         NaN   
Track_Type                                                            Steel   
Undercarriage_Pad_Width                                 None or Unspecified   
Stick_Length                                            None or Unspecified   
Thumb                                                   None or Unspecified   
Pattern_Changer                                         None or Unspecified   
Grouser_Type                                                         Double   
Backhoe_Mounting                                                        NaN   
Blade_Type                                                              NaN   
Travel_Controls                         

Evaluation with RMSLE.

In [43]:
df_raw.SalePrice = np.log(df_raw.SalePrice) # take a log since we want RMSLE

In [22]:
RandomForestRegressor

sklearn.ensemble.forest.RandomForestRegressor

In [21]:
m = RandomForestRegressor(n_jobs=-1)
# RF: Universal ML Technique, data of any kind. 
# It doesn't overfit, separate validation set not needed. No stats assumption, less feature engineering
m.fit(df_raw.drop('SalePrice', axis=1), df_raw.SalePrice)
# SHITF + TAB!!!!!! WHERE HAVE YOU BEEN ALL MY LIFE

ValueError: could not convert string to float: 'Conventional'

Continous variables and categorical variables.

In [25]:
# df_raw.saledate
# From date; Is it a holiday?!? Weekend?!? Raining?!? Sports-Event?! Whats in a date?

In [27]:
fld = df_raw.saledate

In [29]:
# fld.dt.year
# If there is any column and any variation if it - add everything to the dataset!

In [35]:
# polyfil of the function, copied from the repo of fast.ai
def add_datepart(df, fldname):
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower()) # gettattr!
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    df.drop(fldname, axis=1, inplace=True)

In [44]:
add_datepart(df_raw, 'saledate')
df_raw.saleYear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleYear, dtype: int64

In [41]:
df_raw.head()

SalesID  SalePrice  MachineID  ModelID  datasource  auctioneerID  YearMade  \
0  1139246   2.406712     999089     3157         121           3.0      2004   
1  1139248   2.393413     117657       77         121           3.0      1996   
2  1139249   2.220327     434808     7009         121           3.0      2001   
3  1139251   2.356923    1026470      332         121           3.0      2001   
4  1139253   2.230622    1057373    17311         121           3.0      2007   

   MachineHoursCurrentMeter UsageBand fiModelDesc     ...      \
0                      68.0       Low        521D     ...       
1                    4640.0       Low      950FII     ...       
2                    2838.0      High         226     ...       
3                    3486.0      High    PC120-6E     ...       
4                     722.0    Medium        S175     ...       

  saleIs_month_end saleIs_month_start saleIs_quarter_end saleIs_quarter_start  \
0            False              False              False                False   
1            False              False              False                False   
2            False              False              False                False   
3            False              False              False                False   
4            False              False              False                False   

  saleIs_year_end saleIs_year_start saleHour saleMinute saleSecond saleElapsed  
0           False             False        0          0          0  1163635200  
1           False             False        0          0          0  1080259200  
2           False             False        0          0          0  1077753600  
3           False             False        0          0          0  1305763200  
4           False             False        0          0          0  1248307200  

[5 rows x 68 columns]

In [45]:
df_raw.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',


In [48]:
# polyfil of the function, copied from the repo of fast.ai
from pandas.api.types import is_string_dtype, is_numeric_dtype
def train_cats(df):
    for n,c in df.items():
        if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()

In [49]:
train_cats(df_raw)

In [55]:
df_raw.UsageBand.cat.categories # GET FACTORS!!!!! 
# order the categories in sensible order to have a decision

Index(['High', 'Medium', 'Low'], dtype='object')

In [54]:
df_raw.UsageBand.cat.set_categories(['High','Medium', 'Low'], ordered=True, inplace=True) # inplace change the dataframe

In [57]:
# ordinal variables
# missing values %
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

In [63]:
# save to a feather format
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/raw')

In [66]:
# df_raw = pd.read_feather('tmp/raw')

In [71]:
# again polyfill from the fast.ai repo
def numericalize(df, col, name, max_n_cat):
    if not is_numeric_dtype(col) and ( max_n_cat is None or len(col.cat.categories)>max_n_cat):
        df[name] = col.cat.codes+1

def fix_missing(df, col, name, na_dict):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum() or (name in na_dict):
            df[name+'_na'] = pd.isnull(col)
            filler = na_dict[name] if name in na_dict else col.median()
            df[name] = col.fillna(filler)
            na_dict[name] = filler
    return na_dict

def proc_df(df, y_fld=None, skip_flds=None, ignore_flds=None, do_scale=False, na_dict=None,
            preproc_fn=None, max_n_cat=None, subset=None, mapper=None):

    if not ignore_flds: ignore_flds=[]
    if not skip_flds: skip_flds=[]
    if subset: df = get_sample(df,subset)
    else: df = df.copy()
    ignored_flds = df.loc[:, ignore_flds]
    df.drop(ignore_flds, axis=1, inplace=True)
    if preproc_fn: preproc_fn(df)
    if y_fld is None: y = None
    else:
        if not is_numeric_dtype(df[y_fld]): df[y_fld] = df[y_fld].cat.codes
        y = df[y_fld].values
        skip_flds += [y_fld]
    df.drop(skip_flds, axis=1, inplace=True)

    if na_dict is None: na_dict = {}
    else: na_dict = na_dict.copy()
    na_dict_initial = na_dict.copy()
    for n,c in df.items(): na_dict = fix_missing(df, c, n, na_dict)
    if len(na_dict_initial.keys()) > 0:
        df.drop([a + '_na' for a in list(set(na_dict.keys()) - set(na_dict_initial.keys()))], axis=1, inplace=True)
    if do_scale: mapper = scale_vars(df, mapper)
    for n,c in df.items(): numericalize(df, c, n, max_n_cat)
    df = pd.get_dummies(df, dummy_na=True)
    df = pd.concat([ignored_flds, df], axis=1)
    res = [df, y, na_dict]
    if do_scale: res = res + [mapper]
    return res

In [80]:
df = proc_df(df_raw, 'SalePrice')
y = df_raw['SalePrice'].values